# Imports and Downloads

In [ ]:
!pip install -q torch transformers datasets sentence-transformers evaluate tqdm bitsandbytes faiss-cpu activeft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install -q llama-index-postprocessor-flag-embedding-reranker
!pip install -q git+https://github.com/FlagOpen/FlagEmbedding.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
!pip install -q llama-index==0.9.40

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 14.6 MB/s eta 0:00:00


In [ ]:
#Run this cell twice for it to work
import json
import torch
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from typing import List, Dict
from datasets import load_dataset
from llama_index.text_splitter import SentenceSplitter
from llama_index.schema import Document
from activeft.sift import Retriever
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Main Code

In [ ]:
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [ ]:
class Reranker:
    def __init__(self, model_name="BAAI/bge-reranker-large"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name).to(self.device)

    def rerank(self, query: str, passages: List[Dict], top_n: int = 5) -> List[Dict]:
        pairs = [(query, doc['text']) for doc in passages]
        features = self.tokenizer(
            pairs,
            padding=True,
            truncation=True,
            return_tensors='pt',
            max_length=512
        ).to(self.device)

        with torch.no_grad():
            scores = self.model(**features).logits.squeeze().cpu().numpy()

        scored_docs = [(float(score), doc) for score, doc in zip(scores, passages)]
        scored_docs.sort(reverse=True)

        reranked_docs = []
        for score, doc in scored_docs[:top_n]:
            doc_copy = doc.copy()
            doc_copy['score'] = score
            reranked_docs.append(doc_copy)

        return reranked_docs

In [ ]:
class SIFTRetriever:
    def __init__(
        self,
        embedding_model_name='Snowflake/snowflake-arctic-embed-l',
        k=5,
        chunk_size=256,
        lambda_param=0.01,
        also_query_opposite=True,
        rerank=True,
        top_k_rerank=5,
        threads=1,
        noise_std=None
    ):
        self.embedding_model = SentenceTransformer(embedding_model_name)
        self.text_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=25)
        self.k = k
        self.lambda_param = lambda_param
        self.also_query_opposite = also_query_opposite
        self.threads = threads
        self.chunks = []
        self.retriever = None
        self.rerank = rerank
        self.top_k_rerank = top_k_rerank
        self.noise_std = noise_std if noise_std is not None else np.sqrt(lambda_param)

        if rerank:
            self.reranker = Reranker()

In [ ]:
@add_method(SIFTRetriever)
def load_corpus(self, corpus_path: str):
    """Load and prepare corpus for indexing"""
    print("Loading corpus...")
    with open(corpus_path, 'r') as f:
        corpus_data = json.load(f)

    print("Processing documents and creating chunks...")
    for doc in tqdm(corpus_data):
        content = doc.get('body', '')
        if not content:
            continue

        document = Document(
            text=content,
            metadata={
                'title': doc.get('title', ''),
                'source': doc.get('source', ''),
                'published_at': doc.get('published_at', ''),
                'category': doc.get('category', '')
            }
        )

        doc_chunks = self.text_splitter.split_text(document.text)

        for chunk in doc_chunks:
            self.chunks.append({
                'text': chunk,
                'metadata': document.metadata
            })

    self._build_index()


## Run for VTL


In [ ]:
@add_method(SIFTRetriever)
def _build_index(self):
    """Build FAISS index with SIFT retriever"""
    print("Building FAISS index...")
    embeddings = []
    valid_chunks = []

    for chunk in tqdm(self.chunks):
        try:
            embedding = self.embedding_model.encode(chunk['text'], convert_to_tensor=True)
            embeddings.append(embedding.cpu().numpy())
            valid_chunks.append(chunk)
        except Exception as e:
            print(f"Error embedding chunk: {e}")
            continue

    self.chunks = valid_chunks
    embeddings = np.array(embeddings).astype('float32')
    print(f"Generated embeddings shape: {embeddings.shape}")

    faiss.normalize_L2(embeddings)

    dimension = embeddings.shape[1]
    faiss_index = faiss.IndexFlatIP(dimension)  # Using IP (Inner Product) as per SIFT
    faiss_index.add(embeddings)
    print(f"Created FAISS index with {len(embeddings)} embeddings")

    self.retriever = Retriever(
        index=faiss_index,
        llambda=self.lambda_param,
        fast=False,
        also_query_opposite=self.also_query_opposite,
        only_faiss=False
    )

In [ ]:
@add_method(SIFTRetriever)
def retrieve(self, query: str) -> List[Dict]:
    """Retrieve chunks using SIFT with optional reranking"""
    query_embedding = self.embedding_model.encode(query).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)
    faiss.normalize_L2(query_embedding)

    D, I, V, retrieval_time = self.retriever.search(
        query=query_embedding,
        N=self.k if not self.rerank else self.k * 2,  # Retrieve more if reranking
        K=None,
        mean_pooling=False,
        threads=self.threads
    )

    # Handle both single result and array results
    if isinstance(I, np.int64):
        indices = [I]
        scores = [D]
    else:
        indices = I[0] if I.ndim > 1 else I
        scores = D[0] if D.ndim > 1 else D

    results = []
    for idx, score in zip(indices, scores):
        chunk = self.chunks[idx]
        results.append({
            'text': chunk['text'],
            'score': float(score),
            'metadata': chunk['metadata']
        })

    if self.rerank:
        results = self.reranker.rerank(query, results, self.top_k_rerank)

    return results

In [ ]:
def main():
    rerank = True
    top_k = 5
    chunk_size = 256

    # Initialize retriever
    retriever = SIFTRetriever(
        k=20 if rerank else top_k,
        chunk_size=chunk_size,
        rerank=rerank,
        top_k_rerank=top_k
    )

    # Load corpus
    corpus_path = '/content/corpus.json'
    retriever.load_corpus(corpus_path)

    # Load dataset from Hugging Face
    print("Loading dataset from Hugging Face...")
    dataset = load_dataset("yobro4619/multihop_rag_balanced_sample")
    queries = dataset['train']

    results = []
    print("Performing retrieval...")
    for query_item in tqdm(queries):
        retrieved_docs = retriever.retrieve(query_item['query'])

        result = {
            'query': query_item['query'],
            'answer': query_item['answer'],
            'question_type': query_item['question_type'],
            'retrieval_list': retrieved_docs,
            'gold_list': query_item.get('evidence_list', [])
        }
        results.append(result)

    output_file = '/content/enhanced_sift_retrieval.json'
    print(f"Saving results to {output_file}")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

Loading corpus...
Processing documents and creating chunks...


100%|██████████| 609/609 [00:03<00:00, 162.93it/s]


Building FAISS index...


100%|██████████| 6495/6495 [06:09<00:00, 17.58it/s]


Generated embeddings shape: (6495, 1024)
Created FAISS index with 6495 embeddings
Loading dataset from Hugging Face...


Generating train split:   0%|          | 0/24 [00:00<?, ? examples/s]

Performing retrieval...


100%|██████████| 24/24 [01:32<00:00,  3.85s/it]


Saving results to /content/enhanced_sift_retrieval.json


In [ ]:
if __name__ == '__main__':
    main()

## Run for CosineSimilarity

In [ ]:
from activeft.acquisition_functions.cosine_similarity import CosineSimilarity

@add_method(SIFTRetriever)
def _build_index(self):
    """Build FAISS index with SIFT retriever"""
    print("Building FAISS index...")
    embeddings = []
    valid_chunks = []

    for chunk in tqdm(self.chunks):
        try:
            embedding = self.embedding_model.encode(chunk['text'], convert_to_tensor=True)
            embeddings.append(embedding.cpu().numpy())
            valid_chunks.append(chunk)
        except Exception as e:
            print(f"Error embedding chunk: {e}")
            continue

    self.chunks = valid_chunks
    embeddings = np.array(embeddings).astype('float32')
    print(f"Generated embeddings shape: {embeddings.shape}")

    faiss.normalize_L2(embeddings)

    dimension = embeddings.shape[1]
    faiss_index = faiss.IndexFlatIP(dimension)  # Using IP (Inner Product) as per SIFT
    faiss_index.add(embeddings)
    print(f"Created FAISS index with {len(embeddings)} embeddings")

    cosine_similarity = CosineSimilarity(
            target=torch.Tensor(),
            mini_batch_size=1000,
            embedding_batch_size=32
        )

    self.retriever = Retriever(
        index=faiss_index,
        llambda=self.lambda_param,
        fast=False,
        also_query_opposite=self.also_query_opposite,
        only_faiss=False,
        acquisition_function=cosine_similarity
    )

In [ ]:
@add_method(SIFTRetriever)
def retrieve(self, query: str) -> List[Dict]:
    """Retrieve chunks using SIFT with optional reranking"""
    query_embedding = self.embedding_model.encode(query).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)
    faiss.normalize_L2(query_embedding)

    D, I, V, retrieval_time = self.retriever.search(
        query=query_embedding,
        N=self.k if not self.rerank else self.k * 2,  # Retrieve more if reranking
        K=None,
        mean_pooling=False,
        threads=self.threads
    )

    # Handle both single result and array results
    if isinstance(I, np.int64):
        indices = [I]
        scores = [D]
    else:
        indices = I[0] if I.ndim > 1 else I
        scores = D[0] if D.ndim > 1 else D

    results = []
    for idx, score in zip(indices, scores):
        chunk = self.chunks[idx]
        results.append({
            'text': chunk['text'],
            'score': float(score),
            'metadata': chunk['metadata']
        })

    if self.rerank:
        results = self.reranker.rerank(query, results, self.top_k_rerank)

    return results

In [ ]:
def main():
    rerank = True
    top_k = 5
    chunk_size = 256

    # Initialize retriever
    retriever = SIFTRetriever(
        k=20 if rerank else top_k,
        chunk_size=chunk_size,
        rerank=rerank,
        top_k_rerank=top_k
    )

    # Load corpus
    corpus_path = '/content/corpus.json'
    retriever.load_corpus(corpus_path)

    # Load dataset from Hugging Face
    print("Loading dataset from Hugging Face...")
    dataset = load_dataset("yobro4619/multihop_rag_balanced_sample")
    queries = dataset['train']

    results = []
    print("Performing retrieval...")
    for query_item in tqdm(queries):
        retrieved_docs = retriever.retrieve(query_item['query'])

        result = {
            'query': query_item['query'],
            'answer': query_item['answer'],
            'question_type': query_item['question_type'],
            'retrieval_list': retrieved_docs,
            'gold_list': query_item.get('evidence_list', [])
        }
        results.append(result)

    output_file = '/content/sift_.CosineSimilarity.json'
    print(f"Saving results to {output_file}")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

In [ ]:
if __name__ == '__main__':
    main()

Loading corpus...
Processing documents and creating chunks...


100%|██████████| 609/609 [00:03<00:00, 159.70it/s]


Building FAISS index...


100%|██████████| 6495/6495 [05:52<00:00, 18.44it/s]


Generated embeddings shape: (6495, 1024)
Created FAISS index with 6495 embeddings
Loading dataset from Hugging Face...
Performing retrieval...


100%|██████████| 24/24 [01:09<00:00,  2.90s/it]


Saving results to /content/enhanced_sift_retrieval.json


## Run for ITL

In [ ]:
from activeft.acquisition_functions.itl import ITL
from activeft.acquisition_functions.bace import BaCEState

@add_method(SIFTRetriever)
def _build_index(self):
    """Build FAISS index with SIFT retriever"""
    print("Building FAISS index...")
    embeddings = []
    valid_chunks = []

    for chunk in tqdm(self.chunks):
        try:
            embedding = self.embedding_model.encode(chunk['text'], convert_to_tensor=True)
            embeddings.append(embedding.cpu().numpy())
            valid_chunks.append(chunk)
        except Exception as e:
            print(f"Error embedding chunk: {e}")
            continue

    self.chunks = valid_chunks
    embeddings = np.array(embeddings).astype('float32')
    print(f"Generated embeddings shape: {embeddings.shape}")

    faiss.normalize_L2(embeddings)

    dimension = embeddings.shape[1]
    faiss_index = faiss.IndexFlatIP(dimension)  # Using IP (Inner Product) as per SIFT
    faiss_index.add(embeddings)
    print(f"Created FAISS index with {len(embeddings)} embeddings")

    itl = ITL(
            target=torch.Tensor(),  # Will be set during retrieval
            noise_std=self.noise_std,
            mini_batch_size=1000,
        )

    self.retriever = Retriever(
            index=faiss_index,
            llambda=self.lambda_param,
            fast=False,
            also_query_opposite=self.also_query_opposite,
            only_faiss=False,
            acquisition_function=itl
        )

In [ ]:
@add_method(SIFTRetriever)
def retrieve(self, query: str) -> List[Dict]:
    """Retrieve chunks using SIFT with optional reranking"""
    query_embedding = self.embedding_model.encode(query).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)
    faiss.normalize_L2(query_embedding)

    D, I, V, retrieval_time = self.retriever.search(
        query=query_embedding,
        N=self.k if not self.rerank else self.k * 2,  # Retrieve more if reranking
        K=None,
        mean_pooling=False,
        threads=self.threads
    )

    # Handle both single result and array results
    if isinstance(I, np.int64):
        indices = [I]
        scores = [D]
    else:
        indices = I[0] if I.ndim > 1 else I
        scores = D[0] if D.ndim > 1 else D

    results = []
    for idx, score in zip(indices, scores):
        chunk = self.chunks[idx]
        results.append({
            'text': chunk['text'],
            'score': float(score),
            'metadata': chunk['metadata']
        })

    if self.rerank:
        results = self.reranker.rerank(query, results, self.top_k_rerank)

    return results

In [ ]:
def main():
    rerank = True
    top_k = 5
    chunk_size = 256

    # Initialize retriever
    retriever = SIFTRetriever(
        k=20 if rerank else top_k,
        chunk_size=chunk_size,
        rerank=rerank,
        top_k_rerank=top_k
    )

    # Load corpus
    corpus_path = '/content/corpus.json'
    retriever.load_corpus(corpus_path)

    # Load dataset from Hugging Face
    print("Loading dataset from Hugging Face...")
    dataset = load_dataset("yobro4619/multihop_rag_balanced_sample")
    queries = dataset['train']

    results = []
    print("Performing retrieval...")
    for query_item in tqdm(queries):
        retrieved_docs = retriever.retrieve(query_item['query'])

        result = {
            'query': query_item['query'],
            'answer': query_item['answer'],
            'question_type': query_item['question_type'],
            'retrieval_list': retrieved_docs,
            'gold_list': query_item.get('evidence_list', [])
        }
        results.append(result)

    output_file = '/content/sift_.itl.json'
    print(f"Saving results to {output_file}")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

In [ ]:
if __name__ == '__main__':
    main()

Loading corpus...
Processing documents and creating chunks...


100%|██████████| 609/609 [00:03<00:00, 165.72it/s]


Building FAISS index...


100%|██████████| 6495/6495 [05:54<00:00, 18.30it/s]


Generated embeddings shape: (6495, 1024)
Created FAISS index with 6495 embeddings
Loading dataset from Hugging Face...
Performing retrieval...


100%|██████████| 24/24 [01:27<00:00,  3.63s/it]


Saving results to /content/sift_.itl.json


## Run for LazyVTL

In [ ]:
from typing import Tuple
from activeft.model import ModelWithEmbeddingOrKernel
class PatchedLazyVTL(LazyVTL):
  def select_from_minibatch(
          self,
          batch_size: int,
          model: ModelWithEmbeddingOrKernel | None,
          data: torch.Tensor,
          device: torch.device | None,
      ) -> Tuple[torch.Tensor, torch.Tensor]:
          state = self.initialize(model, data, device)

          # Debugging: Check sizes
          print(f"Priority queue size: {self.priority_queue.size() if self.priority_queue else 'None'}")
          print(f"Data size: {data.size(0)}")

          # Reinitialize the priority queue if the size does not match
          if self.priority_queue is None or self.priority_queue.size() != data.size(0):
              print("Reinitializing priority queue to match data size...")
              self.set_initial_priority_queue(
                  embeddings=data,
                  target_embedding=torch.mean(state.target, dim=0),
              )

          assert (
              self.priority_queue is not None
          ), "Initial priority queue must be set."
          assert self.priority_queue.size() == data.size(
              0
          ), f"Size of the priority queue ({self.priority_queue.size()}) must match the size of the data set ({data.size(0)})."

          selected_values = []
          for _ in range(batch_size):
              while True:
                  i, _ = self.priority_queue.pop()
                  new_value, new_covariance_matrix, state = self.recompute(state, i)

                  prev_top_value = self.priority_queue.top_value
                  self.priority_queue.push(i, new_value)
                  if new_value >= prev_top_value:
                      break
              selected_values.append(new_value)
              state = self.step(state, i, new_covariance_matrix)
          return torch.tensor(state.selected_indices), torch.tensor(selected_values)

In [ ]:
from activeft.acquisition_functions.lazy_vtl import LazyVTL
from activeft.acquisition_functions.bace import BaCEState
import os

@add_method(SIFTRetriever)
def _build_index(self, save_path="faiss_index.bin", load_if_exists=True):
    """Build FAISS index with SIFT retriever and save it to disk."""
    if load_if_exists and os.path.exists(save_path):
        print(f"Loading FAISS index from {save_path}...")
        self.faiss_index = faiss.read_index(save_path)
        print(f"Loaded FAISS index with {self.faiss_index.ntotal} embeddings.")
    else:
        print("Building FAISS index...")
        embeddings = []
        valid_chunks = []

        for chunk in tqdm(self.chunks):
            try:
                embedding = self.embedding_model.encode(chunk['text'], convert_to_tensor=True)
                embeddings.append(embedding.cpu().numpy())
                valid_chunks.append(chunk)
            except Exception as e:
                print(f"Error embedding chunk: {e}")
                continue

        self.chunks = valid_chunks
        embeddings = np.array(embeddings).astype('float32')
        print(f"Generated embeddings shape: {embeddings.shape}")

        faiss.normalize_L2(embeddings)

        dimension = embeddings.shape[1]
        faiss_index = faiss.IndexFlatIP(dimension)  # Using IP (Inner Product) as per SIFT
        faiss_index.add(embeddings)
        print(f"Created FAISS index with {len(embeddings)} embeddings")

        # Save the FAISS index to disk
        faiss.write_index(faiss_index, save_path)
        print(f"Saved FAISS index to {save_path}")

        self.faiss_index = faiss_index

    # Initialize LazyVTL instead of ITL
    lazy_vtl = PatchedLazyVTL(
        target=torch.Tensor(),  # Will be set during retrieval
        noise_std=self.noise_std,
        mini_batch_size=1000,
    )

    self.retriever = Retriever(
        index=self.faiss_index,
        llambda=self.lambda_param,
        fast=False,
        also_query_opposite=self.also_query_opposite,
        only_faiss=False,
        acquisition_function=lazy_vtl  # Use LazyVTL as the acquisition function
    )

In [ ]:
@add_method(SIFTRetriever)
def retrieve(self, query: str) -> List[Dict]:
    """Retrieve chunks using SIFT with optional reranking"""
    query_embedding = self.embedding_model.encode(query).astype('float32')
    query_embedding = query_embedding.reshape(1, -1)
    faiss.normalize_L2(query_embedding)

    D, I, V, retrieval_time = self.retriever.search(
        query=query_embedding,
        N=self.k if not self.rerank else self.k * 2,  # Retrieve more if reranking
        K=None,
        mean_pooling=False,
        threads=self.threads
    )

    # Handle both single result and array results
    if isinstance(I, np.int64):
        indices = [I]
        scores = [D]
    else:
        indices = I[0] if I.ndim > 1 else I
        scores = D[0] if D.ndim > 1 else D

    results = []
    for idx, score in zip(indices, scores):
        chunk = self.chunks[idx]
        results.append({
            'text': chunk['text'],
            'score': float(score),
            'metadata': chunk['metadata']
        })

    if self.rerank:
        results = self.reranker.rerank(query, results, self.top_k_rerank)

    return results

In [ ]:
def main():
    rerank = True
    top_k = 5
    chunk_size = 256

    # Initialize retriever
    retriever = SIFTRetriever(
        k=20 if rerank else top_k,
        chunk_size=chunk_size,
        rerank=rerank,
        top_k_rerank=top_k
    )

    # Load corpus
    corpus_path = '/content/corpus.json'
    retriever.load_corpus(corpus_path)

    # Load dataset from Hugging Face
    print("Loading dataset from Hugging Face...")
    dataset = load_dataset("yobro4619/multihop_rag_balanced_sample")
    queries = dataset['train']

    results = []
    print("Performing retrieval...")
    for query_item in tqdm(queries):
        retrieved_docs = retriever.retrieve(query_item['query'])

        result = {
            'query': query_item['query'],
            'answer': query_item['answer'],
            'question_type': query_item['question_type'],
            'retrieval_list': retrieved_docs,
            'gold_list': query_item.get('evidence_list', [])
        }
        results.append(result)

    output_file = '/content/sift_.LazyVTL.json'
    print(f"Saving results to {output_file}")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)

In [ ]:
if __name__ == '__main__':
    main()

Loading corpus...
Processing documents and creating chunks...


100%|██████████| 609/609 [00:03<00:00, 162.38it/s]


Building FAISS index...


100%|██████████| 6495/6495 [06:27<00:00, 16.74it/s]


Generated embeddings shape: (6495, 1024)
Created FAISS index with 6495 embeddings
Saved FAISS index to faiss_index.bin
Loading dataset from Hugging Face...
Performing retrieval...


...

Priority queue size: 280
Data size: 1000
Reinitializing priority queue to match data size...
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 495
Reinitializing priority queue to match data size...
Priority queue size: 495
Data size: 280
Reinitializing priority queue to match data size...


 96%|█████████▌| 23/24 [06:19<00:13, 13.87s/it]

Priority queue size: 280
Data size: 1000
Reinitializing priority queue to match data size...
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 1000
Priority queue size: 1000
Data size: 495
Reinitializing priority queue to match data size...
Priority queue size: 495
Data size: 280
Reinitializing priority queue to match data size...


100%|██████████| 24/24 [06:32<00:00, 16.37s/it]


Saving results to /content/sift_.LazyVTL.json
